# Sensors measures

## 0. Clean Data (Open Refine)

## 1. Read Files

In [23]:
import pandas as pd
import os

csv_files = list()
json_files = list()

rel_path="../data/raw/"
for root, dirs, files in os.walk(rel_path):
    for filename in files:
        if filename.split('.')[1] == 'csv':
            
            csv_files.append({"data":pd.read_csv(rel_path + filename, parse_dates =["time"], index_col ="time"),"type":filename.split('-')[0]})
        else:
            json_files.append({"data":pd.read_json(rel_path + filename,lines=True,convert_dates=['time']),"type":filename.split('-')[0]})
print(csv_files[:1])
print(json_files[:1])

[{'data':                             sensor  value
time                                     
2017-12-22 11:22:11+00:00  T-DHT11     27
2017-12-22 11:22:16+00:00  T-DHT11     28
2017-12-22 11:22:20+00:00  T-DHT11     28
2017-12-22 11:22:24+00:00  T-DHT11     28
2017-12-22 11:22:28+00:00  T-DHT11     28
...                            ...    ...
2018-03-13 11:41:53+00:00  T-DHT11     26
2018-03-13 11:41:57+00:00  T-DHT11     26
2018-03-13 11:42:01+00:00  T-DHT11     26
2018-03-13 11:42:05+00:00  T-DHT11     26
2018-03-13 11:42:09+00:00  T-DHT11     26

[1420308 rows x 2 columns], 'type': 'T'}]
[{'data':     sensor  value                      time
0  T-DM280  26.43 2017-12-22 10:51:31+00:00
1  T-DM280  26.43 2017-12-22 10:51:35+00:00
2  T-DM280  26.45 2017-12-22 10:51:38+00:00
3  T-DM280  26.45 2017-12-22 10:51:41+00:00, 'type': 'T'}]


## 2. Clean Data
### 2.1. Rename columns with representable values

In [32]:
for df in (csv_files+json_files):
    if df['type']=='H':
        df['data'].rename(columns={"value":"humidity"}, inplace=True)
    if df['type'] == 'T':
        df['data'].rename(columns={"value":"temperature"}, inplace=True)
    if df['type'] == 'P':
        df['data'].rename(columns={"value":"hPa"}, inplace=True)
    df['data']

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1420308 entries, 2017-12-22 11:22:11+00:00 to 2018-03-13 11:42:09+00:00
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   sensor       1420308 non-null  object
 1   temperature  1420308 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 32.5+ MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1428377 entries, 2017-12-20 12:17:05+00:00 to 2018-03-13 11:42:12+00:00
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   sensor    1428377 non-null  object 
 1   humidity  1428377 non-null  float64
dtypes: float64(1), object(1)
memory usage: 32.7+ MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1422795 entries, 2017-12-22 10:52:59+00:00 to 2018-03-13 11:42:08+00:00
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 

### Convert the time to index

In [39]:
for df in json_files:
    df['data'].set_index('time')